In [3]:
import pandas as pd
from meta_project.data.data_loader import DataLoader
data_loader = DataLoader()

df = data_loader.load_and_merge_data()
df.head()


,id,image_name,image_id,class_id,class_name,is_training_image
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1,001.Black_footed_Albatross,0
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,2,1,001.Black_footed_Albatross,1
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,3,1,001.Black_footed_Albatross,0
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,4,1,001.Black_footed_Albatross,1
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,5,1,001.Black_footed_Albatross,1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
import os
import time
import psutil # Do monitorowania pamięci
import gc     # Garbage collector

# --- Konfiguracja ---
ROOT_DIR = os.path.join(os.getcwd(), "data", "raw", "CUB_200_2011", "images")
# !!! KLUCZOWY PARAMETR - ZACZNIJ OD MAŁEJ WARTOŚCI !!!
BATCH_SIZE = 8  # Spróbuj 8, jeśli nadal za dużo, zmniejsz do 4, 2 lub 1
LEARNING_RATE = 1e-4
EPOCHS = 10 # Zwiększ dla lepszych wyników, ale najpierw ustabilizuj pamięć
TEST_SPLIT_RATIO = 0.25
NUM_WORKERS = 2 # Liczba procesów do ładowania danych. Zacznij od 0 lub 2.
                # Zwiększaj ostrożnie, bo też zużywają RAM. 0 = ładowanie w głównym procesie.

# Sprawdź, czy ścieżka istnieje
if not os.path.exists(ROOT_DIR):
    raise FileNotFoundError(f"Dataset directory not found: {ROOT_DIR}. "
                         "Please ensure the CUB_200_2011 dataset is downloaded and extracted correctly.")

# --- Przygotowanie urządzenia (GPU lub CPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device == torch.device("cuda"):
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# --- Transformacje Danych ---
# Użyjemy tych samych transformacji co poprzednio
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Transformacje dla zbioru treningowego (z augmentacją)
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
])

# Transformacje dla zbioru walidacyjnego/testowego (bez augmentacji)
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

# --- Ładowanie Danych ---
print("Loading dataset using ImageFolder...")
# ImageFolder oczekuje struktury: root/class/image.jpg
# Automatycznie znajdzie klasy i przypisze indeksy
full_dataset = datasets.ImageFolder(root=ROOT_DIR)
class_names = full_dataset.classes
num_classes = len(class_names)

if num_classes == 0:
    raise ValueError(f"No classes found in {ROOT_DIR}. Check dataset structure.")
if num_classes != 200:
     print(f"Warning: Expected 200 classes, but found {num_classes}. Check dataset structure.")

print(f"Found {len(full_dataset)} images in {num_classes} classes.")

# --- Podział na Zbiór Treningowy i Testowy ---
total_size = len(full_dataset)
test_size = int(TEST_SPLIT_RATIO * total_size)
train_size = total_size - test_size

print(f"Splitting into Train ({train_size}) and Test ({test_size}) sets...")
# Ustawiamy generator dla powtarzalności podziału
generator = torch.Generator().manual_seed(42)
# Domyślnie ImageFolder nie ma atrybutu 'targets', musimy go dodać lub użyć innej metody podziału
# Prostsza metoda: załaduj z różnymi transformacjami
train_dataset = datasets.ImageFolder(root=ROOT_DIR, transform=transform_train)
test_dataset_with_test_transform = datasets.ImageFolder(root=ROOT_DIR, transform=transform_test)

# Teraz dzielimy indeksy, a nie same datasety
indices = list(range(total_size))
train_indices, test_indices = random_split(indices, [train_size, test_size], generator=generator)

# Tworzymy Subsets z odpowiednimi transformacjami
from torch.utils.data import Subset
train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset_with_test_transform, test_indices)

print("Dataset split complete.")

# --- Tworzenie DataLoaderów ---
# DataLoader będzie ładował dane w batchach
# pin_memory=True może przyspieszyć transfer na GPU (jeśli używasz CUDA)
# Ustaw `persistent_workers=True` jeśli `NUM_WORKERS > 0` dla potencjalnej poprawy szybkości między epokami
use_persistent_workers = NUM_WORKERS > 0

print(f"Creating DataLoaders with Batch Size: {BATCH_SIZE}, Num Workers: {NUM_WORKERS}")
train_loader = DataLoader(
    train_subset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=use_persistent_workers
)
test_loader = DataLoader(
    test_subset,
    batch_size=BATCH_SIZE, # Można użyć większego batcha do testów, jeśli RAM pozwala
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=use_persistent_workers
)
print("DataLoaders created.")

# --- Definicja Modelu ---
print("Loading ResNet18 model...")
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Zamrożenie wag (opcjonalnie, na początku transfer learningu)
# for param in model.parameters():
#     param.requires_grad = False

# Modyfikacja ostatniej warstwy (klasyfikatora)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
print(f"Modified final layer for {num_classes} classes.")

# Przeniesienie modelu na odpowiednie urządzenie
model.to(device)
print("Model moved to device.")

# --- Definicja Funkcji Straty i Optymalizatora ---
criterion = nn.CrossEntropyLoss()
# Optymalizujemy tylko parametry, które wymagają gradientu (na wypadek zamrożenia)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
print("Loss function and optimizer defined.")

# --- Pętla Treningowa ---
print(f"\n--- Starting Training for {EPOCHS} Epochs ---")
print(f"Initial RAM Usage: {psutil.virtual_memory().percent}% Used")
if device == torch.device("cuda"):
     print(f"Initial GPU Memory: {torch.cuda.memory_allocated(device)/1024**2:.2f} MB Allocated, {torch.cuda.memory_reserved(device)/1024**2:.2f} MB Reserved")


for epoch in range(EPOCHS):
    start_time_epoch = time.time()
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    # --- Faza Treningu ---
    model.train() # Ustawienie modelu w tryb treningu
    running_loss = 0.0
    batches_processed_train = 0

    for i, (inputs, labels) in enumerate(train_loader):
        start_time_batch = time.time()
        # Przeniesienie danych na urządzenie
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Wyzerowanie gradientów
        optimizer.zero_grad()

        # Przejście w przód (forward pass)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Przejście wstecz (backward pass) i optymalizacja
        loss.backward()
        optimizer.step()

        # Statystyki
        running_loss += loss.item()
        batches_processed_train += 1

        if (i + 1) % 20 == 0: # Loguj co 20 batchy
            avg_loss_recent = running_loss / batches_processed_train # Średnia od początku epoki
            batch_time = time.time() - start_time_batch
            print(f'  Batch [{i+1:>4}/{len(train_loader):>4}] Loss: {avg_loss_recent:.4f} | Time/Batch: {batch_time:.3f}s')
            # Reset running loss for recent average if needed, or keep accumulating epoch loss
            # running_loss = 0.0 # If you want loss for only last 20 batches
            # batches_processed_train = 0

        # Agresywne czyszczenie pamięci (użyj jeśli masz problemy)
        del inputs, labels, outputs, loss
        if device == torch.device("cuda"):
             torch.cuda.empty_cache()
        # gc.collect() # gc.collect() może być wolne, używaj oszczędnie

    epoch_loss_train = running_loss / len(train_loader)

    # --- Faza Walidacji ---
    model.eval() # Ustawienie modelu w tryb ewaluacji
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad(): # Wyłączenie obliczania gradientów
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Opcjonalne czyszczenie
            del inputs, labels, outputs, loss, predicted
            if device == torch.device("cuda"):
                 torch.cuda.empty_cache()
            # gc.collect()

    epoch_loss_val = val_loss / len(test_loader)
    epoch_acc_val = 100 * correct / total if total > 0 else 0
    end_time_epoch = time.time()
    epoch_duration = end_time_epoch - start_time_epoch

    print("-" * 50)
    print(f"Epoch {epoch+1} Summary:")
    print(f"  Train Loss: {epoch_loss_train:.4f}")
    print(f"  Val Loss:   {epoch_loss_val:.4f}")
    print(f"  Val Acc:    {epoch_acc_val:.2f}%")
    print(f"  Duration:   {epoch_duration:.2f}s")
    print(f"  RAM Usage: {psutil.virtual_memory().percent}% Used")
    if device == torch.device("cuda"):
        print(f"  GPU Memory: {torch.cuda.memory_allocated(device)/1024**2:.2f} MB Allocated, {torch.cuda.memory_reserved(device)/1024**2:.2f} MB Reserved")
    print("-" * 50)

    # Wywołaj garbage collector na koniec epoki
    gc.collect()
    if device == torch.device("cuda"):
        torch.cuda.empty_cache()


print("\n--- Training Finished ---")

# --- (Opcjonalnie) Zapisanie modelu ---
# print("Saving model...")
# torch.save(model.state_dict(), "cub200_resnet18_pytorch.pth")
# print("Model saved to cub200_resnet18_pytorch.pth")

Using device: cuda
GPU Name: NVIDIA GeForce RTX 3090 Ti
Loading dataset using ImageFolder...
Found 11788 images in 200 classes.
Splitting into Train (8841) and Test (2947) sets...
Dataset split complete.
Creating DataLoaders with Batch Size: 8, Num Workers: 2
DataLoaders created.
Loading ResNet18 model...


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Michal/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 37.4MB/s]


Modified final layer for 200 classes.
Model moved to device.
Loss function and optimizer defined.

--- Starting Training for 10 Epochs ---
Initial RAM Usage: 41.6% Used
Initial GPU Memory: 43.08 MB Allocated, 66.00 MB Reserved

Epoch 1/10


: 